
#  Game Analysis

##  Heroes of Pymoli
* Of the 576 players engaging in in-game purchases, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Setup

In [1]:
#  Dependencies
import pandas as pd
import numpy as np


In [2]:
#  Get source data for analysis
sourcefile = "Resources/purchase_data.csv"
raw_csv = pd.read_csv(sourcefile)

#  Preview source data
raw_csv.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


### Player Count

In [3]:
#  Identify the number of purchases made by each player who engaged in a transaction
player_purchases = raw_csv["SN"].value_counts()

#  Identify total number of players who engaged in a transaction
player_count = pd.DataFrame({"Total Players" : [len(player_purchases)]})
player_count

,Total Players
0,576


### Summary of Purchase Data

In [4]:
#  ID which types of items were bought
items_bought = raw_csv["Item ID"].value_counts()     # NOTE: there are 4 items that have unique ID's despite similar names!

#  ID how many different items were bought
items_count = len(items_bought)

#  Average amount spent on a purchase
avg_price = raw_csv["Price"].mean()

#  Get the number of total transactions
trans_count = len(raw_csv["Purchase ID"].unique())

#  Determine the total amount 
total_value = raw_csv["Price"].sum()

#  Summarize findings
purchase_summary = pd.DataFrame({"Number of Unique Items":[items_count], 
                                 "Average Price":["${:,.2f}".format(avg_price)],
                                 "Number of Purchases":[trans_count],
                                 "Total Revenue":["${:,.2f}".format(total_value)]
                                })
purchase_summary

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


### Gender Demographics Data

In [5]:
#  Get gender breakdowns on purchases
gender_grp = raw_csv.groupby(["Gender"])

#  Get unique ID's within the gender groups
gender_SNs = gender_grp["SN"].unique()

#  Figure out the counts of purchasers for each gender
gender_count = [len(i) for i in gender_SNs]

#  Gather results into a new data frame
gender_data = pd.DataFrame(gender_SNs)
gender_data["Total Count"] = gender_count

#  Add in percentage information
gender_data["Percentage in Decimal"] = gender_data["Total Count"] / gender_data["Total Count"].sum()

#  Formated version of the percentage information
gender_data["Percentage of Players"] = ["{:,.2f}%".format(i * 100) for i in gender_data["Percentage in Decimal"]]

#  Summarize findings
gender_dem_summ = gender_data[["Total Count", "Percentage of Players"]].sort_values(["Total Count"], ascending=False)
gender_dem_summ

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


### Gender Purchase Analysis 

In [6]:
#  Number of purchases by gender allowing for multiple purchases from the same ID
gender_data["Purchase Count"] = gender_grp["Purchase ID"].count()

#  Total spend of each gender group
gender_data["Total Spend"] = gender_grp["Price"].sum()

#  Average purchase price by gender
gender_data["Average Price"] = gender_data["Total Spend"] / gender_data["Purchase Count"]

#  Average amount spent by gender
gender_data["Average Spend"] = gender_data["Total Spend"] / gender_data["Total Count"] 

#  Format numeric results for readability
gender_data["Average Purchase Price"] = ["${:,.2f}".format(i) for i in gender_data["Average Price"]]
gender_data["Total Purchase Value"] = ["${:,.2f}".format(i) for i in gender_data["Total Spend"]]
gender_data["Avg Total Purchase per Person"] = ["${:,.2f}".format(i) for i in gender_data["Average Spend"]]

#  Summarize findings
gender_purch_summ = gender_data[["Purchase Count", 
                            "Average Purchase Price", 
                            "Total Purchase Value", 
                            "Avg Total Purchase per Person"
                           ]].sort_values(["Purchase Count"], ascending=False)
gender_purch_summ

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


### Age demographics data

In [7]:
#  Bins for age groups
#   "Round" number age groups
round_bin = [0, 9.9167, 14.9167, 19.9167, 24.9167, 29.9167, 34.9167, 39.9167, 120]
round_lbl = ["Under 10", "10 - 14", "15 - 19", "20 - 24", "25 - 29",  "30 - 34", "35 - 39", "40+"]

#   Age grouped on generation
cur_yr = 2018
gen_bin = [0,
           cur_yr - 2013, # Gen Alpha
           cur_yr - 1995, # Gen Z
           cur_yr - 1980, # Millenials
           cur_yr - 1985, # Gen Y
           cur_yr - 1965, # Gen X
           cur_yr - 1946, # Baby Boomers
           cur_yr - 1925, # Silent Gen
           cur_yr - 1910, # Greatest Gen
           cur_yr - 1901  # Interbellum
           ]
gen_lbl = ["Gen Alpha", "Gen Z", "Millenials", "Gen Y", "Gen X", 
           "Baby Boomers", "Silent Gen", "Greatest Gen", "Interbellum"]

#   Ages around school/work milestones
mile_bin = [0, 
              11, # Grade school
              13, # Middle school
              17, # High school
              21, # Undergrad
              25, # Post-grad
              30, # Young Adult
              55, # Adult
              65, # AARP eligible -_-
              120 # Senior
             ]
mile_lbl = ["Grade School", "Middle School", "High School", 
            "Undergrads", "Post-Grads", "Young Adults", 
            "Adults", "AARP Eligible", "Seniors"]



In [8]:
#  Add age range to a copy of the data based on desired bins and labels
#   Edit here for alternative groupings
age_demo = raw_csv
age_demo["Age Range"] = pd.cut(age_demo["Age"], round_bin, labels=round_lbl)

#  Group the data by the new age range category
age_demo = age_demo.groupby("Age Range")

In [9]:
#  Count the number of unique user handles in each age range into a data frame
age_data = pd.DataFrame(age_demo["SN"].unique())
age_count = [len(i) for i in age_data["SN"]]
age_data["Total Count"] = age_count

#  Add in percentage measures
age_data["Pct Calc"] = age_data["Total Count"] / age_data["Total Count"].sum()

#  Format numbers
age_data["Percentage of Players"] = ["{:,.2f}%".format(i * 100) for i in age_data["Pct Calc"]]

#  Extract Summary Information
age_demo_summ = age_data[["Total Count", "Percentage of Players"]]

#  Display results
age_demo_summ

,Total Count,Percentage of Players
Age Range,,
Under 10,17,2.95%
10 - 14,22,3.82%
15 - 19,107,18.58%
20 - 24,258,44.79%
25 - 29,77,13.37%
30 - 34,52,9.03%
35 - 39,31,5.38%
40+,12,2.08%


In [ ]:
#  Identify top spenders

In [ ]:
#  Identify most popular items

In [ ]:
#  Identify most profitable items